In [3]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import cv2
import math
import time
import libraryQ4 as lib
pylab.rcParams['figure.figsize'] = (20,10)
%matplotlib

## PROGRAMME STARTS

# Pre-annotated ground truths
ground = {0: np.array([[435,   6, 167, 194]]), 1: np.array([[198, 143, 191, 173]]), 2:
np.array([[ 97, 101,  98,  80]]), 3: np.array([[322, 150,  69,  69]]), 4:
np.array([[175, 104, 172, 180]]), 5: np.array([[426, 141,  92,  97]]), 6:
np.array([[210, 120,  63,  57]]), 7: np.array([[241, 174, 127, 131]]), 8:
np.array([[840, 225, 123, 104],[ 67, 255,  59,  84]]), 9:
np.array([[188,  36, 257, 253]]), 10: np.array([[ 78, 101, 121, 113],
[578, 127,  60,  91],[916, 149,  37,  69]]), 11:
np.array([[170, 107,  68,  50]]), 12: np.array([[153,  74,  65, 146]]), 13:
np.array([[269, 126, 135, 130]]), 14:
np.array([[117, 106, 134, 113],[981, 101, 128, 113]]), 15:
np.array([[151,  56, 136, 131]])}

# Setting the thresholds
edgethresh = 2
judgethresh = 0.4
# Setting the max and min radius of a detected circle in HT
minrad = 10
maxrad = 100
# Set the min proximity of any 2 HT circles`
proximity = 70

# whichdartimgs = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
whichdartimgs = [9]

F1VJ = {}
F1VJHT = {}
PrecisionVJ = {}
PrecisionVJHT = {}
RecallVJ = {}
RecallVJHT = {}

start = time.time()
i=9

# Loading a given image
location = str("../images/dart") + str(i) + str(".jpg")
imgcol = cv2.imread(location)
img = cv2.cvtColor(imgcol, cv2.COLOR_BGR2GRAY)
print ("dart" + str(i) + ".jpg loaded")
if len(whichdartimgs) == 1:
    lib.imshow(imgcol, "Original Image.")


# Finding the edges of the image above a threshold
stime = time.time()
grad, direc = lib.EdgeDetect (img, threshavg=edgethresh)
print ("EdgeDetect runtime: " + str(time.time() - stime) )

# Saving the edge image
saveloc = (str("detectedtest/dart" + str(i) + str("edge.jpg")))
cv2.imwrite(saveloc,grad)
print ("Edge image saved")
if len(whichdartimgs) == 1:
    plt.title ("Edge image. Click to close")
    plt.imshow (grad, cmap='gray')
    plt.waitforbuttonpress()
    plt.close()

# Finding abd labeling the detected dartboards for Viola-Jones
stime = time.time()
classifier = cv2.CascadeClassifier('cascade.xml')
dart_VJ = lib.ViolaJones(i, imgcol, classifier)
newgrad = np.zeros(grad.shape)
for (x,y,w,h) in dart_VJ:
        for xx in range (x,x+w):
            for yy in range (y, y+h):
                newgrad[yy,xx] = grad[yy,xx]
        cv2.rectangle(imgcol, (x,y), (x+w,y+h), (0,165,255), 3)
plt.imshow(newgrad, cmap='gray')
grad = newgrad
plt.waitforbuttonpress()
# Saving VJ detected colour image
saveloc = (str("detectedtest/dart" + str(i) + str("VJ_detect.jpg")))
cv2.imwrite(saveloc,imgcol)
print ("Viola-Jones Runtime: " + str(time.time()-stime))
print ("Viola-Jones image saved")
if len(whichdartimgs) == 1:
    lib.imshow(imgcol, "Viola-Jones detection.")

#Reloading a fresh coloured image
imgcol = cv2.imread(location)

#Running the Hough Transform for circles
stime = time.time()
Hxyr = lib.HTCircle(grad, direc, minrad, maxrad)
etime = time.time()
print("runtime: Hough Transform " + str(etime-stime))

# Finding the Hough Space for the Hough Transform
stime = time.time()
Hspace = lib.HSpace(Hxyr)

#Saving the Hough Space image
saveloc = (str("detectedtest/dart" + str(i) + str("HS.jpg")))
cv2.imwrite(saveloc,Hspace)
print ("Hough Space Runtime: " + str(time.time()-stime))
print ("Hough image saved")
if len(whichdartimgs) == 1:
    plt.title ("Hough Space. Click to close")
    plt.imshow(Hspace, cmap='gray')
    plt.waitforbuttonpress()
    plt.close()


Using matplotlib backend: Qt5Agg
dart9.jpg loaded
EdgeDetect runtime: 4.372434377670288
Edge image saved
Viola-Jones Runtime: 1.6690170764923096
Viola-Jones image saved
Performing Hough Transform - Circles:
progress - 0%
progress - 3%
progress - 7%
progress - 10%
progress - 14%
progress - 17%
progress - 20%
progress - 24%
progress - 27%
progress - 31%
progress - 34%
progress - 37%
progress - 41%
progress - 44%
progress - 47%
progress - 51%
progress - 54%
progress - 58%
progress - 61%
progress - 64%
progress - 68%
progress - 71%
progress - 75%
progress - 78%
progress - 81%
progress - 85%
progress - 88%
progress - 92%
progress - 95%
progress - 98%
runtime: Hough Transform 81.91424107551575
Hough Space Runtime: 14.072793006896973
Hough image saved


In [49]:
%matplotlib
def PlotRectangle (imgcol, Hxyr, Hspace, minrad, maxrad, prox=50):
    nc =  maxrad-minrad #num circles
    (a,b) = Hspace.shape #for later to calculate the highest index of the flattened array
    argordH = Hspace.argsort(axis=None) #lists the indices in accending order of value (array is flattened)

    index = 1
    #in order to detect images with multiple dart boards, we find the top highest Hspace values that dont have

#circle0
    (y0,x0)=np.unravel_index(argordH[a*b-index], (a,b)) #unravels the flattened value back into a tuple for the nth highest
    amax = Hxyr[y0,x0].shape
    argordxyr = Hxyr[y0,x0].argsort(axis=None)
    print (argordxyr)
    
    tt = 5
    
    for h in range (1,tt):
        rindex = argordxyr[amax[0]*1-h]
        r = (rindex+minrad+1)*maxrad/nc
        circle0 = (x0,y0,r)
        cv2.circle(imgcol, (x0,y0), int(r), (255,165,0) )
        
    argordmax = []
    for rr in range (1,10):
        rindex = argordxyr[amax[0]*1-rr]
        argordmax.append(rindex)
    argmax = []
    for rr in argordmax:
        argmax.append(Hxyr[y0,x0][rr])
    print ("ARGMAX = " + str(argmax))
    if max(argordmax)-min(argordmax)<10:
        rindex = argordxyr[amax[0]*1-1]
        r = (rindex+minrad+1)*maxrad/nc
        circle0 = (x0-int(r),y0-int(r),int(2*r),int(2*r))
    else:
        circle0 = (0,0,0,0)
        

    index += 1
    for c in range(index, a*b):
        (y,x)=np.unravel_index(argordH[a*b-c], (a,b)) #unravels the flattened value back into a tuple for the nth highest
        if math.sqrt((x0-x)**2 + (y0-y)**2)>prox:
            index = c
            break

    (y1,x1)=np.unravel_index(argordH[a*b-index], (a,b)) #unravels the flattened value back into a tuple for the nth highest
    amax = Hxyr[y1,x1].shape
    argordxyr = Hxyr[y1,x1].argsort(axis=None)
    print (argordxyr)
    rindex = argordxyr[amax[0]*1-1]
    r = (rindex+minrad+1)*maxrad/nc

    for h in range (1,tt):
        rindex = argordxyr[amax[0]*1-h]
        r = (rindex+minrad+1)*maxrad/nc
        circle1 = (x1,y1,r)
        cv2.circle(imgcol, (x1,y1), int(r),(255,0,0))
    
    argordmax = []
    for rr in range (1,10):
        rindex = argordxyr[amax[0]*1-rr]
        argordmax.append(rindex)
    argmax = []
    for rr in argordmax:
        argmax.append(Hxyr[y1,x1][rr])
    print ("ARGMAX = " + str(argmax))
    if max(argordmax)-min(argordmax)<10:
        rindex = argordxyr[amax[0]*1-1]
        r = (rindex+minrad+1)*maxrad/nc
        circle1 = (x1-int(r),y1-int(r),int(2*r),int(2*r))
    else:
        circle1 = (0,0,0,0)    
    
#circle2
    for c in range(index, a*b):
        (y,x)=np.unravel_index(argordH[a*b-c], (a,b)) #unravels the flattened value back into a tuple for the nth highest
        if math.sqrt((x0-x)**2 + (y0-y)**2)>prox and math.sqrt((x1-x)**2 + (y1-y)**2)>prox:
            index = c
            break

    (y2,x2)=np.unravel_index(argordH[a*b-index], (a,b)) #unravels the flattened value back into a tuple for the nth highest
    amax = Hxyr[y2,x2].shape
    argordxyr = Hxyr[y2,x2].argsort(axis=None)
    print (argordxyr)
    rindex = argordxyr[amax[0]*1-1]
    r = (rindex+minrad+1)*maxrad/nc

    for h in range (1,tt):
        rindex = argordxyr[amax[0]*1-h]
        r = (rindex+minrad+1)*maxrad/nc
        circle2 = (x2,y2,r)
        cv2.circle(imgcol, (x2,y2), int(r), (255,165,0))
        
    argordmax = []
    for rr in range (1,10):
        rindex = argordxyr[amax[0]*1-rr]
        argordmax.append(rindex)
    argmax = []
    for rr in argordmax:
        argmax.append(Hxyr[y2,x2][rr])
    print ("ARGMAX = " + str(argmax))
    if max(argordmax)-min(argordmax)<10:
        rindex = argordxyr[amax[0]*1-1]
        r = (rindex+minrad+1)*maxrad/nc
        circle2 = (x2-int(r),y2-int(r),int(2*r),int(2*r))
    else:
        circle2 = (0,0,0,0)
        
    return circle0, circle1, circle2

Using matplotlib backend: Qt5Agg


In [50]:



# Finding the rectangle that encloses the 3 most likely circles
rect0, rect1, rect2 = PlotRectangle(imgcol, Hxyr, Hspace, minrad, maxrad, prox=proximity)
dart_HT = np.array([rect0,rect1,rect2])

lib.imshow(imgcol)

# Plotting the HT detection on the coloured image
for (x,y,w,h) in dart_HT:
        cv2.rectangle(imgcol, (x,y), (x+w,y+h), (255,165,0), 3)

# Saving the HT detected colour image
saveloc = (str("detectedtest/dart" + str(i) + str("HS_detect.jpg")))
cv2.imwrite(saveloc,imgcol)
print ("Hough transform image saved")
if len(whichdartimgs) == 1:
    lib.imshow(imgcol, "Hough Transform detection.")


# Reload the coloured image
imgcol = cv2.imread(location)

# Combining Viola-Jones and Hough Transform by finding the overlapping classifications and plotting the corresponding VJ rectangle
_, dart_VJHT = lib.Eval(dart_VJ,dart_HT, imgcol, thresh=judgethresh)
# Note: the judgement array is unused

# Saving the detection of combined VJ and HT
saveloc = (str("detectedtest/dart" + str(i) + str("VJHS_detect.jpg")))
cv2.imwrite(saveloc,imgcol)
print ("Joint HT & VJ image saved")
if len(whichdartimgs) == 1:
    lib.imshow(imgcol, "Joint HT & VJ detection.")

# Reload the coloured image
imgcol = cv2.imread(location)
judgementVJHT, _ =lib.Eval(ground[i], dart_VJHT, imgcol, thresh=judgethresh)
imgcol = cv2.imread(location)
judgementVJ, _ = lib.Eval(ground[i], dart_VJ,imgcol, thresh=judgethresh)
detectionVJHT = lib.getinfo(judgementVJHT,dart_VJHT)
detectionVJ = lib.getinfo(judgementVJ,dart_VJ)
F1VJHT[i] = lib.f1score(detectionVJHT)
F1VJ[i] = lib.f1score(detectionVJ)
PrecisionVJ[i] = lib.ppv(detectionVJ)
PrecisionVJHT[i] = lib.ppv(detectionVJHT)
RecallVJ[i] = lib.tpr(detectionVJ)
RecallVJHT[i] = lib.tpr(detectionVJHT)
print ("dart" + str(i) + ".jpg done")
print ("Total runtime: " + str(time.time()-start))
print ("F1-VJ:")
print (F1VJ)
print ("F1-VJHT:")
print (F1VJHT)

# avgVJ_P = sum(PrecisionVJ.values())/len(PrecisionVJ)
# avgVJ_R = sum(RecallVJ.values())/len(RecallVJ)
# avgVJ_F1 = 2*avgVJ_P*avgVJ_R/(avgVJ_P + avgVJ_R)
# print ("VJ - Mean F1: " + str(avgVJ_F1))

# avgVJHT_R = sum(RecallVJHT.values())/len(RecallVJHT)
# avgVJHT_P = sum(PrecisionVJHT.values())/len(PrecisionVJHT)
# avgVJHT_F1 = 2*avgVJHT_R*avgVJHT_P/(avgVJHT_P + avgVJHT_R)
# print ("VJHT - Mean F1: " + str(avgVJHT_F1))

# lib.f1bar(F1VJ.values(), F1VJHT.values(), whichdartimgs)


[ 0 61 60 59 58 57 55 54 53 52 50 48 46 45 88 42 62 41 63 65 87 86 85 84
 83 82 81 80 79 77 76 75 74 70 68 64 37 89 25 22 23 24 36 26 27 28  7  6
 21  5  8 31 32 34 35  3 19 78 69 17 12 66 44 56 51 30 47 33 43 40 20 72
 73 38 71 13 29 67 39 18 11 16  4  9 49 14  1  2 15 10]
ARGMAX = [24.0, 19.0, 10.0, 8.0, 8.0, 7.0, 7.0, 6.0, 5.0]
[44 66 65 64 63 62 61 59 67 58 56 51 50 45 88 43 42 57 68 69 70 87 86 85
 84 83 82 81 80 79 78 77 76 75 74 73 72 71 41 40 89 38 22 21 20 19 18 17
 16 15 14 13 12 11 10  9  7  6  4  2  1 39 24 23 33 27 37 28 29 30 31 36
 35 34 32 25  3 26 49 52  5 60  8 55  0 48 46 54 47 53]
ARGMAX = [33.0, 30.0, 22.0, 21.0, 8.0, 5.0, 5.0, 3.0, 2.0]
[89 88 84 48 49 50 53 54 55 56 57 59 60 61 62 63 64 65 80 79 78 77 76 75
 81 74 72 71 70 68 83 66 73 37 82  9 18 21 22 87 24 10 86 85 17  8 14 30
  2  6  3  5  7  4 47 69 38 40 52 51 31 32 58 11 13 35 34 25 28 39 43 41
  1 33 29 42 36 16 19 15 20 12 46  0 67 23 44 45 26 27]
ARGMAX = [11.0, 9.0, 9.0, 8.0, 7.0, 6.0, 6.0, 5.0, 5.0]
Ho